In [1]:
import os
os.chdir("..")
os.chdir("src")
from data_cleaning import DataCleaning
import polars as pl
os.chdir("..")
import nltk
from unidecode import unidecode
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/bruno/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
cleaning_pipeline = DataCleaning()
df = cleaning_pipeline.run()

2024-03-25 19:00:45 - INFO - Data loaded!
2024-03-25 19:00:45 - INFO - Data types asserted!
2024-03-25 19:00:45 - INFO - Null values cleaned!
2024-03-25 19:00:45 - INFO - Full data cleaned!


In [3]:
df.collect()

CD_ORGAO,NM_ORGAO,ANO_LICITACAO,DS_OBJETO,VL_LICITACAO,DT_HOMOLOGACAO,VL_HOMOLOGADO
i64,str,i64,str,f64,date,f64
88024,"""PM DE BOZANO""",2017,"""Contratação de…",10200.0,2017-02-14,10200.0
45001,"""CM DE CAXIAS D…",2017,"""Fornecimento d…",5498.1,2017-01-26,5498.1
56200,"""PM DE RONDINHA…",2017,"""CESSÃO ONEROSA…",186000.0,2017-02-10,186500.0
66600,"""PM DE DOUTOR M…",2017,"""AQUISIÇÃO DE Ó…",8469.0,2017-01-31,5007.5
66600,"""PM DE DOUTOR M…",2017,"""AQUISIÇÃO DE G…",18781.29,2017-01-31,16265.55
…,…,…,…,…,…,…
54900,"""PM DE PORTO AL…",2021,"""Concurso da se…",150000.0,2022-03-08,150000.0
1204,"""SUSEPE - SUPER…",2021,"""Caracterização…",12862.45,2021-04-12,12600.0
54907,"""DMAE - DEP. MU…",2021,"""Picareta""",6978.0,2022-02-14,7999.0


# 3. **Feature Engineering**

Three features will be created to assist in the analysis:

- tp_orgao: Indicates the type of public agency, there are a total of 17 types
- dt_homologacao_mes: Number of the month in which the bidding was approved
- dt_homologacao_semana_do_ano: Number of the week of the year in which the bidding was approved

## 3.1. **tp_orgao**

In [4]:
nm_orgao_list = [item[0] for item in df.select("NM_ORGAO").collect().rows()]

In [5]:
stop_words = stopwords.words('portuguese')

In [6]:
def remove_stop_words(words):
    return [word for word in words if word.lower() not in stop_words]

In [7]:
from typing import AnyStr
def preprocess(x: AnyStr) -> str:

    special_chars = "¨'!#$%&()*+,./:;<=>?@[\]^_`{|}~"
    new_x = x.replace('"',' ')
    for c in special_chars:
        new_x = new_x.replace(c,' ') # Removes special characters
    new_x = re.sub(r'[^\w\s]', ' ', new_x) # Removes punctuation
    new_x = re.sub("http\S+", ' ',new_x) # Removes links
    new_x = re.sub("@\w+", ' ',new_x) # Removes @
    new_x = re.sub('#\S+', ' ',new_x) # Removes hashtags
    new_x = re.sub('[0-9]+', ' ',new_x) # Removes numbers
    new_x = unidecode(new_x) # Removes accents
    new_x = re.sub("\s+", ' ',new_x) # Removes spaces
    new_x = new_x.strip()
    return new_x


In [8]:
df = df.with_columns(
    pl.col("NM_ORGAO")
    .map_elements(lambda x: nltk.word_tokenize(x.lower(), language='portuguese')) # Tokenize
    .map_elements(remove_stop_words, return_dtype=pl.List(pl.String)) # Removes stop words
    .map_elements(lambda x: [preprocess(word) for word in x]) # Other preprocessing
    .map_elements(lambda arr: list(filter(None, arr))) # Removes items with none
    .alias("NM_ORGAO_NLP")
)

In [9]:
def contains_substring(lst, substrings):
    for item in lst:
        if any(substring in item for substring in substrings):
            return True
    return False



In [ ]:
[item[0] for item in df.select("NM_ORGAO").collect().rows()]

## FALTA VER PQ NAO TA CONVERTENDO PRA LISTA

In [13]:
city_halls = [item[0] for item in df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["pm", "pref", "prefeitura"])
    )
).select(pl.col("NM_ORGAO_NLP")).collect().rows]

TypeError: 'method' object is not iterable

In [14]:
city_halls.collect().to_array()

<LazyFrame [1 col, {"NM_ORGAO_NLP": Unknown}] at 0x74233EE56EF0>

In [10]:
# City Halls
city_halls = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["pm", "pref", "prefeitura"])
    )
).select(pl.col("NM_ORGAO_NLP"))

# City Councils
city_councils = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["cm", "camara"])
    )
).select(pl.col("NM_ORGAO_NLP"))

# Police
police = ['POLÍCIA CIVIL', 'BRIGADA MILITAR',
          'IGP - INSTITUTO-GERAL DE PERÍCIAS',
          'PC - POLÍCIA CIVIL',

          ]

# Banks
banks = ['BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL',
          'BANCO DO ESTADO DO RIO GRANDE DO SUL S.A.',
          'BANRISUL CARTÕES S.A.',
          'BANRISUL S.A. CORRETORA DE VALORES MOBILIÁRIOS E CÂMBIO',
          'BANRISUL S.A. - ADMINISTRADORA DE CONSÓRCIOS',
          'BADESUL DESENVOLVIMENTO S.A. - AGÊNCIA DE FOMENTO/RS',
          'BANRISUL CORRETORA DE SEGUROS S.A.',
          'BRDE - BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL',
          'BANRISUL - BANCO DO ESTADO DO RIO GRANDE DO SUL S.A.',
          'BANRISUL SOLUÇÕES EM PAGAMENTOS S.A. - INSTITUIÇÃO DE PAGAMENTO',
          


          ]

# # Social Security and Welfare Institutions
# social_security = ['INST. PREV. ASSIST. SERV. MUN. DE CANOAS',
#          'FUND. DE ASSIST. SOCIAL - CAXIAS DO SUL',
#          'INST. DE PREV. E ASSIST. DOS SERV. MUNIC. DE N. HAMBURGO',
#          'INST. PREV. ASSIST. MUN. - CAXIAS DO SUL',
#          'INST. PREV. SERV. PUBL. MUNIC. DE IJUÍ',
#          'INST. PREV.  ASSIST. À SAÚDE SERV. PÚBL. MUN. DE S. MARIA',
#          'INSTITUTO ERECHINENSE DE PREVIDÊNCIA - IEP',
#          'SISTEMA DE PREVIDÊNCIA MUNICIPAL - S. LIVRAMENTO',
#          'FUNDAÇÃO DE PREVIDÊNCIA COMPLEMENTAR DO SERV. PÚBLIC. DO ESTADO DO RS',
#          'INST. PREV. DOS SERV. PÚBL. MUN. DE SÃO GABRIEL',
#          'FUND. DE ASSIST. SOCIAL E CIDADANIA - PORTO ALEGRE',
#          'INST. PREV. ASSIST. SERV. MUN. CAMPO BOM',
#          'INST. APOS. PENSÕES SERV. MUN. - SÃO LEOPOLDO',
#          'INST. PREV. DOS SERV. PÚBL. DO MUNIC. - TRÊS PASSOS',
#          'FUND. GAÚCHA DO TRABALHO E AÇÃO SOCIAL',
#          'PREV-XANGRI-LÁ',
#          'FUND. DE ATENDIMENTO SÓCIO-EDUCATIVO DO RGS',
#          'DEP. MUNICIPAL DE HABITAÇÃO - PORTO ALEGRE',
#          'INST. PREV. SERV. PÚBL. MUN. DE CACHOEIRINHA - IPREC',
#          'FUND. DE PROTEÇÃO ESPECIAL DO RIO GRANDE DO SUL',
#          'FUND. VENÂNCIO RAMOS DA SILVA - ITAQUI',
#          'INST. MUN. ASSIST. SERV. DE NOVA SANTA RITA'
#          'IPREV - INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DE VIAMÃO',
#          'URUPREV - URUGUAIANA PREVIDENCIA SOCIAL',
#          'STAS - SECRETARIA DE TRABALHO E ASSISTÊNCIA SOCIAL',
         

# ]
# Social Security and Welfare Institutions
social_security = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["pev", "previdencia", "iprev", "assistencia", "acao"])
    )
).select(pl.col("NM_ORGAO_NLP"))

# social_security = [item[0] for item in df.filter(pl.col("NM_ORGAO_NLP").str.contains_any(["pev", "previdencia", "iprev", "assistencia", "acao"])).select(pl.col("NM_ORGAO_NLP")).collect().rows()]

# Health Care institutions
# health_care = ['FUND. DE SAÚDE PÚBLICA DE NOVO HAMBURGO',
#         'FUND. HOSPITAL MUNICIPAL GETÚLIO VARGAS',
#         'HOSP. MUNICIPAL SÃO JOSÉ DE IBIRAIARAS',
#         'HOSP. NOSSA SRA. DO CARMO - AUTARQUIA MUNICIPAL - TAPES',
#         'CONS. INTERM. DE SAÚDE DO VALE DO RIO TAQUARI',
#         'CONS. INTERM. SAÚDE DO NOROESTE DO ESTADO DO RS',
#         'FUND. HOSPITALAR SANTA TEREZINHA DE ERECHIM',
#         'CONS. INTERM. DE SAÚDE - CONISA',
#         'FUND. DE SAÚDE PÚBLICA SÃO CAMILO DE ESTEIO - FSPSCE',
#         'CONSORCIO DE SAUDE INTER-MUNICIPAL',
#         'CONS. INTERM. DE SAÚDE DA REGIÃO DAS MISSÕES',
#         'SECRETARIA DA SAÚDE DO RS',
#         'INST. MUN. DE ESTRAT.  DE SAÚDE DA FAMÍLIA - IMESF',
#         'CAIXA PREST. ASSIST. SERV. SAÚDE DOS SERV. MUN. DE P. FUNDO',
#         'FHGV - FUNDAÇÃO HOSPITALAR GETÚLIO VARGAS',
#         'SES - SECRETARIA DA SAÚDE DO RS',

#         ]

health_care = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["hospital", "hospitalar", "saude", "hosp"])
    )
).select(pl.col("NM_ORGAO_NLP"))


# Energy companies
energy = ['CENTRAIS ELÉTRICAS DE CARAZINHO S/A.',
           'CIA. DE GÁS DO ESTADO DO RIO GRANDE DO SUL',
           'DEP. MUNICIPAL DE ENERGIA - IJUÍ',
           'DEP. MUNICIPAL DE ENERGIA DE IJUÍ - DEMEI GERAÇÃO']

# Transportation companies
transportation = ['CIA. CARRIS PORTO-ALEGRENSE',
              'EMPRESA PÚBLICA DE TRANSPORTE E CIRCULAÇÃO S/A',
              'DEP. AUTÁRQUICO TRANSP. COLETIVOS - RIO GRANDE',
              'DEP. ESTADUAL DE TRÂNSITO',
              'FUND. PASSO FUNDO DE TURISMO PASSOTUR',
              'EMPRESA GAÚCHA DE RODOVIAS S.A.']

# # Water and Sewer companies
# water_sewer = ['SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL',
#                'COMUSA - SERVIÇOS DE ÁGUA E ESGOTO DE NOVO HAMBURGO',
#                'DEP. DE ÁGUA E ESGOTOS - SANT\'ANA DO LIVRAMENTO',
#                'SERVIÇO AUTÔN. DE SANEAMENTO DE PELOTAS',
#                'CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RIO DOS SINOS',
#                'DEP. MUNICIPAL DE ÁGUAS E SANEAMENTO DE IJUÍ - DEMASI',
#                'FUND. MUNICIPAL DE SAÚDE DE CANOAS',
#                'FUND. MÉD.-HOSP. DR. HONOR T. COSTA - LAVRAS DO SUL',
#                'CIA. RIOGRANDENSE DE SANEAMENTO',
#                'DEPARTAMENTO DE ÁGUA, ARROIOS E ESGOTO DE BAGÉ',
#                'DEP. MUNICIPAL DE ÁGUA E ESGOTOS - PORTO ALEGRE',
#                'EMPRESA FARROUPILHENSE DE SANEAMENTO E DESENVOLVIMENTO AMBIENTAL S.A.',
#                'ÁGUAS DA NASCENTE - SERV. MUN. DE SANEAMENTO AMBIENTAL',
#                'ÁGUA DE IVOTI',
#                'TERMASA - CIA. ÁGUAS TERMAIS MARCELINO RAMOS',
#                'SEMAE - SERVIÇO MUN. DE ÁGUA E ESGOTOS - S. LEOPOLDO',
#                'DEMASI - DEP. MUNICIPAL DE ÁGUAS E SANEAMENTO DE IJUÍ',
#                'DMAE - DEP. MUNICIPAL DE ÁGUA E ESGOTOS - PORTO ALEGRE',
#                'DAEB - DEPARTAMENTO DE ÁGUA, ARROIOS E ESGOTO DE BAGÉ',
#                'DAE - DEP. DE ÁGUA E ESGOTOS - SANTANA DO LIVRAMENTO',
#                'SAMAE - SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL',
#                ]

water_sewer = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["agua", "esgoto", "aguas", "esgotos", "saneamento"])
    )
).select(pl.col("NM_ORGAO_NLP"))


# Universities
universities = ['UNIVERSIDADE ESTADUAL DO RIO GRANDE DO SUL',
                'UERGS - UNIVERSIDADE ESTADUAL DO RIO GRANDE DO SUL',
                ]

# Intermunicipal Associations
intermunicipal_associations = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["cons", "consorcio", "consorcios"])
    )
).select(pl.col("NM_ORGAO_NLP"))


# intermunicipal_associations = ['CONS. INTERM. DO VALE DO JACUÍ - CI/JACUÍ'
#               'CONS. PÚBL. INTERM. P/ ASSUNTOS ESTRATÉGICOS DO G8 - CIPAE G8',
#               'CONS. INTERM. - CI - CENTRO SUL',
#               'CONS. INTERM. ABRIGO INSTITUCIONAL DA PAZ - CIAIP',
#               'CONS. INTERM. DA REGIÃO CENTRO DO ESTADO DO RS - CI/CENTRO STA. MARIA',
#               'CONS. INTERM. DA REGIÃO NORDESTE',
#               'CONS. INTERM. DE COOP. EM GESTÃO PÚBLICA - CONIGEPU',
#               'CONS. INTERM. DE DESENV. SUSTENT. DA SERRA GAÚCHA - CISGA',
#               'CONS. INTERM. DE GESTÃO AMPLIADA DA REGIÃO CARBONÍFERA - CIGA',
#               'CONS. INTERM. DE SERVIÇOS DO VALE DO RIO PARDO',
#               'CONS. INTERM. DO MÉDIO ALTO URUGUAI',
#               'CONS. INTERM. DO VALE DO RIO CAÍ - CIS/CAÍ',
#               'CONS. INTERM. ESMERALDA / PINHAL DA SERRA',
#               'CONS. INTERM. LAR DE ACOLHIMENTO SÃO FRANCISCO',
#               'ADM. DE  CONSÓRCIOS  INTERMUNIC. S/A. - CAXIAS DO SUL',
#               'CONS. PÚBL. INTERM. DA REGIÃO DO ALTO URUGUAI - CIRAU',
#               'CONS. PÚBL. INTERM. DESENV. ECON., SOC. E AMB. MUN. BACIA RIO JAGUARÃO',
#               'CONS. DE DESENV. INTERM. DOS MUNIC. DO ALTO JACUÍ',
#               'CONS. DE DESENV. SUSTENT. DA QUARTA COLÔNIA - CONDESUS/QUARTA COLÔNIA',
#               'CONS. PÚBL. DA ASSOC. DOS MUNIC. DA REGIÃO METR. DE POA - CP GRANPAL',
#               'CONS. PÚBL. DO EXTREMO SUL', 'CONS. PÚBL. FRONTEIRA NOROESTE',
#               'CONSÓRCIO INTERMUNICIPAL DA REGIÃO DO PLANALTO MÉDIO',
#               'CONSÓRCIO PÚBLICO DA ASSOCIAÇÃO DOS MUNICÍPIOS DO VALE DO RIO DOS SINO',
#               'CONSÓRCIO PÚBLICO INTERMUNICIPAL DE INCLUSÃO SOCIAL E PRODUTIVA',
#               'CI/CENTRO - CONS. INTERM. DA REGIÃO CENTRO  ESTADO DO RS - STA. MARIA',
#               'COMANDAÍ - CONS. INTERM. DE GERENCIAMENTO DE RESÍDUOS SÓLIDOS URBANOS',
#               'CIEPS - CONS. INTERM. ESMERALDA / PINHAL DA SERRA',
#               'CONIGEPU - CONS. INTERM. DE COOP. EM GESTÃO PÚBLICA',
#               'CONISA - CONS. INTERM. DE SAÚDE',
#               'CIMAU - CONS. INTERM. DO MÉDIO ALTO URUGUAI',
#              'CIGRES - CONS. INTERM. DE GESTÃO DE RESÍDUOS SÓLIDOS',
#              'CISA - CONS. INTERM. SAÚDE DO NOROESTE DO ESTADO DO RS',
#              'CONSISA - CONS. INTERM. DE SERVIÇOS DO VALE DO TAQUARI',
#              'CIGA - CONS. INTERM. DE GESTÃO AMPLIADA DA REGIÃO CARBONÍFERA',
#              'CI-CS - CONS. INTERM. - CI - CENTRO SUL',
#              'CIS/CAÍ - CONS. INTERM. DO VALE DO RIO CAÍ',
#              'CIRENOR - CONS. INTERM. DA REGIÃO NORDESTE',
#              'CISMISSÕES - CONS. INTERM. DE SAÚDE DA REGIÃO DAS MISSÕES',
#              'CISGS - CONS. INTERM. DE SAÚDE DO GRANDE SARANDI',
#              'CONDESUS - CONS. INTERM. DE DESENV. SUSTENT. CAMINHO DAS ORIGENS',
#              'CI/JACUÍ - CONS. INTERM. DO VALE DO JACUÍ',
#              'CONDESUS - CONS. INTERM. DE DESENV. SUSTENT.- REGIÃO CAMPOS CIMA SERRA',
#              'CITEGEM - CONS. INTERM. DE GESTÃO MULTIFUNCIONAL',
#              'CIAIP - CONS. INTERM. ABRIGO INSTITUCIONAL DA PAZ',
#              'CILASFRA - CONS. INTERM. LAR DE ACOLHIMENTO SÃO FRANCISCO',
#              'CRESU - CONS. INTERM. DE RESÍDUOS SÓLIDOS',
#              'SDR - SECRETARIA DE DESENVOLVIMENTO RURAL',
#              'CISGA - CONS. INTERM. DE DESENV. SUSTENT. DA SERRA GAÚCHA - CISGA',
#              'COIS - CONS. INTERM. DE SAÚDE',
#              'CISVALE - CONS. INTERM. DE SERVIÇOS DO VALE DO RIO PARDO',
#              'CIRAU - CONS. PÚBL. INTERM. DA REGIÃO DO ALTO URUGUAI - CIRAU',
#              'CISP - CONSÓRCIO PÚBLICO INTERMUNICIPAL DE INCLUSÃO SOCIAL E PRODUTIVA',
#              'COMAJA - CONS. DE DESENV. INTERM. DOS MUNIC. DO ALTO JACUÍ',
#              'CP GRANPAL - CONS. PÚBL. DA ASSOC. DOS MUNIC. DA REGIÃO METR. DE POA',
#              'CIPLAM - CONSÓRCIO INTERMUNICIPAL DA REGIÃO DO PLANALTO MÉDIO',
#              'CP AMLINOR - CONS. PÚBL. DA ASSOC. DOS MUNIC. DO LITORAL NORTE',
#              'YUCUMÃ - CONS. ROTA DO YUCUMÃ',
#              'CONSIM - CONSORCIO DE SAUDE INTER-MUNICIPAL',
#              'COMUNORS - CONSÓRCIO DE DESENV. DOS MUNICÍPIOS DO NORTE DO RS',
#              'CIPAE G8 - CONS. PÚBL. INTERM. P/ ASSUNTOS ESTRATÉGICOS DO G8',
#              'COFRON - CONS. PÚBL. FRONTEIRA NOROESTE',
#              'CP SINOS - CONS. PÚBLICO DA ASSOC. MUNICÍPIOS DO VALE RIO DOS SINOS',
#              'CONDESUS/QUARTA COLÔNIA - CONS. DE DESENV. SUSTENT. DA QUARTA COLÔNIA',
#              'PRÓ-SINOS - CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RIO DOS SINOS',
#              'ADCOINTER - ADM. DE  CONSÓRCIOS  INTERMUNIC. S/A. - CAXIAS DO SUL',
#              'CPES - CONS. PÚBL. DO EXTREMO SUL',
#              'CONREPAR - CONS. REGIONAL DO PARANHANA',]


# Regulatory Agencies
regulatory_agencies = ['AGÊNCIA EST. DE REGULAÇÃO DOS SERV. PÚBL. DELEG. DO RGS',
                  'AGÊNCIA MUN. REGULAÇÃO  DOS SERV. PÚBL. DELEG. DE S. GABRIEL',
                  'AGÊNCIA REGULADORA DE SERVIÇOS PÚBLICOS DELEGADOS DE SANTA CRUZ DO SUL',
                  'AGÊNCIA REGULADORA DOS SERV. PÚBL. MUNIC. DE ERECHIM']

# Technology Agencies
technology_agencies = ['CIA. DE PROCESSAMENTO DE DADOS DO RS',
              'EMPRESA MUNICIPAL DE INFORMÁTICA DE PELOTAS',
              'CIA. PROC. DE DADOS DO MUNIC. PORTO ALEGRE']

# Cultural Organizations
cultural_organizations = ['FENAC S.A. FEIRAS E EMPREEND. TURÍSTICOS - N. HAMBURGO',
           'CIA. ÁGUAS TERMAIS MARCELINO RAMOS - TERMASA',
           'FESTA NACIONAL DA UVA TURISMO E EMPREENDIMENTOS S/A.',
           'FUND. CULT. AFIF JORGE SIMÕES FILHO - SÃO SEPÉ',
           'FUND. MUN. DE ARTES DE MONTENEGRO',
           'CIA. RIOGRANDENSE DE ARTES GRÁFICAS',
           'SEL - SECRETARIA DO ESPORTE E LAZER',
           'FUNDARC - FUND. MUN. DE ARTE E CULTURA - GRAVATAÍ',
           'FCAJSF - FUND. CULT. AFIF JORGE SIMÕES FILHO - SÃO SEPÉ',          
           ]

# Agricultural Institutions
agricultural_institutions = ['ASSOCIAÇÃO SULINA DE CRÉDITO E ASSISTÊNCIA RURAL',
               'CIA. ESTADUAL DE SILOS E ARMAZÉNS',
               'CIA. RIOGRANDENSE DE MINERAÇÃO',
               'INSTITUTO RIOGRANDENSE DO ARROZ',
               'FUND. MUN. DE MEIO AMBIENTE - GRAVATAÍ']

# State Agencies (Secretarias do Estado, Tribunais, Assembléia, etc)
state_agencies = ['PROCURADORIA-GERAL DE JUSTIÇA',
                     'SECRETARIA DA FAZENDA DO RS',
                     'SECRETARIA DA SEGURANÇA PÚBLICA',
                     'SECRETARIA DE JUSTIÇA, CIDADANIA E DIREITOS HUMANOS',
                     'SECRETARIA DO MEIO AMBIENTE E INFRAESTRUTURA',
                     'TRIBUNAL DE CONTAS DO ESTADO DO RS',
                     'TRIBUNAL DE JUSTIÇA MILITAR',
                     'DEFENSORIA PÚBLICA DO ESTADO',
                     'ASSEMBLEIA LEGISLATIVA']

# Other Institutions/Organizations
others = ['SUPERINTENDÊNCIA DO PORTO DE RIO GRANDE',
          'CENTRAL DE LICITAÇÕES DO ESTADO',
          'CENTRAIS DE ABASTECIMENTO DO RIO GRANDE DO SUL S.A.',
          'CIA. DE DESENV. DE CAXIAS DO SUL',
          'CIA. DE DESENV. DE PASSO FUNDO',
          'CIA. INDUSTRIAL E DE DESENV. URBANO - CRUZ ALTA',
          'INST. DE PLANEJAMENTO DE SANTA MARIA',
          'CIA. MUNICIPAL DE URBANISMO - NOVO HAMBURGO',
          'CODEPAS-P. - CIA. DE DESENV. DE PASSO FUNDO']

In [11]:
# City Halls
city_halls = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["pm", "pref", "prefeitura"])
    )
).select(pl.col("NM_ORGAO_NLP"))

# City Councils
city_councils = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["cm", "camara"])
    )
).select(pl.col("NM_ORGAO_NLP"))

# Police
police = ['POLÍCIA CIVIL', 'BRIGADA MILITAR',
          'IGP - INSTITUTO-GERAL DE PERÍCIAS',
          'PC - POLÍCIA CIVIL',

          ]

# Banks
banks = ['BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL',
          'BANCO DO ESTADO DO RIO GRANDE DO SUL S.A.',
          'BANRISUL CARTÕES S.A.',
          'BANRISUL S.A. CORRETORA DE VALORES MOBILIÁRIOS E CÂMBIO',
          'BANRISUL S.A. - ADMINISTRADORA DE CONSÓRCIOS',
          'BADESUL DESENVOLVIMENTO S.A. - AGÊNCIA DE FOMENTO/RS',
          'BANRISUL CORRETORA DE SEGUROS S.A.',
          'BRDE - BANCO REGIONAL DE DESENVOLVIMENTO DO EXTREMO SUL',
          'BANRISUL - BANCO DO ESTADO DO RIO GRANDE DO SUL S.A.',
          'BANRISUL SOLUÇÕES EM PAGAMENTOS S.A. - INSTITUIÇÃO DE PAGAMENTO',
          


          ]

# # Social Security and Welfare Institutions
# social_security = ['INST. PREV. ASSIST. SERV. MUN. DE CANOAS',
#          'FUND. DE ASSIST. SOCIAL - CAXIAS DO SUL',
#          'INST. DE PREV. E ASSIST. DOS SERV. MUNIC. DE N. HAMBURGO',
#          'INST. PREV. ASSIST. MUN. - CAXIAS DO SUL',
#          'INST. PREV. SERV. PUBL. MUNIC. DE IJUÍ',
#          'INST. PREV.  ASSIST. À SAÚDE SERV. PÚBL. MUN. DE S. MARIA',
#          'INSTITUTO ERECHINENSE DE PREVIDÊNCIA - IEP',
#          'SISTEMA DE PREVIDÊNCIA MUNICIPAL - S. LIVRAMENTO',
#          'FUNDAÇÃO DE PREVIDÊNCIA COMPLEMENTAR DO SERV. PÚBLIC. DO ESTADO DO RS',
#          'INST. PREV. DOS SERV. PÚBL. MUN. DE SÃO GABRIEL',
#          'FUND. DE ASSIST. SOCIAL E CIDADANIA - PORTO ALEGRE',
#          'INST. PREV. ASSIST. SERV. MUN. CAMPO BOM',
#          'INST. APOS. PENSÕES SERV. MUN. - SÃO LEOPOLDO',
#          'INST. PREV. DOS SERV. PÚBL. DO MUNIC. - TRÊS PASSOS',
#          'FUND. GAÚCHA DO TRABALHO E AÇÃO SOCIAL',
#          'PREV-XANGRI-LÁ',
#          'FUND. DE ATENDIMENTO SÓCIO-EDUCATIVO DO RGS',
#          'DEP. MUNICIPAL DE HABITAÇÃO - PORTO ALEGRE',
#          'INST. PREV. SERV. PÚBL. MUN. DE CACHOEIRINHA - IPREC',
#          'FUND. DE PROTEÇÃO ESPECIAL DO RIO GRANDE DO SUL',
#          'FUND. VENÂNCIO RAMOS DA SILVA - ITAQUI',
#          'INST. MUN. ASSIST. SERV. DE NOVA SANTA RITA'
#          'IPREV - INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DE VIAMÃO',
#          'URUPREV - URUGUAIANA PREVIDENCIA SOCIAL',
#          'STAS - SECRETARIA DE TRABALHO E ASSISTÊNCIA SOCIAL',
         

# ]
# Social Security and Welfare Institutions
social_security = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["pev", "previdencia", "iprev", "assistencia", "acao"])
    )
).select(pl.col("NM_ORGAO_NLP"))

# social_security = [item[0] for item in df.filter(pl.col("NM_ORGAO_NLP").str.contains_any(["pev", "previdencia", "iprev", "assistencia", "acao"])).select(pl.col("NM_ORGAO_NLP")).collect().rows()]

# Health Care institutions
# health_care = ['FUND. DE SAÚDE PÚBLICA DE NOVO HAMBURGO',
#         'FUND. HOSPITAL MUNICIPAL GETÚLIO VARGAS',
#         'HOSP. MUNICIPAL SÃO JOSÉ DE IBIRAIARAS',
#         'HOSP. NOSSA SRA. DO CARMO - AUTARQUIA MUNICIPAL - TAPES',
#         'CONS. INTERM. DE SAÚDE DO VALE DO RIO TAQUARI',
#         'CONS. INTERM. SAÚDE DO NOROESTE DO ESTADO DO RS',
#         'FUND. HOSPITALAR SANTA TEREZINHA DE ERECHIM',
#         'CONS. INTERM. DE SAÚDE - CONISA',
#         'FUND. DE SAÚDE PÚBLICA SÃO CAMILO DE ESTEIO - FSPSCE',
#         'CONSORCIO DE SAUDE INTER-MUNICIPAL',
#         'CONS. INTERM. DE SAÚDE DA REGIÃO DAS MISSÕES',
#         'SECRETARIA DA SAÚDE DO RS',
#         'INST. MUN. DE ESTRAT.  DE SAÚDE DA FAMÍLIA - IMESF',
#         'CAIXA PREST. ASSIST. SERV. SAÚDE DOS SERV. MUN. DE P. FUNDO',
#         'FHGV - FUNDAÇÃO HOSPITALAR GETÚLIO VARGAS',
#         'SES - SECRETARIA DA SAÚDE DO RS',

#         ]

health_care = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["hospital", "hospitalar", "saude", "hosp"])
    )
).select(pl.col("NM_ORGAO_NLP"))


# Energy companies
energy = ['CENTRAIS ELÉTRICAS DE CARAZINHO S/A.',
           'CIA. DE GÁS DO ESTADO DO RIO GRANDE DO SUL',
           'DEP. MUNICIPAL DE ENERGIA - IJUÍ',
           'DEP. MUNICIPAL DE ENERGIA DE IJUÍ - DEMEI GERAÇÃO']

# Transportation companies
transportation = ['CIA. CARRIS PORTO-ALEGRENSE',
              'EMPRESA PÚBLICA DE TRANSPORTE E CIRCULAÇÃO S/A',
              'DEP. AUTÁRQUICO TRANSP. COLETIVOS - RIO GRANDE',
              'DEP. ESTADUAL DE TRÂNSITO',
              'FUND. PASSO FUNDO DE TURISMO PASSOTUR',
              'EMPRESA GAÚCHA DE RODOVIAS S.A.']

# # Water and Sewer companies
# water_sewer = ['SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL',
#                'COMUSA - SERVIÇOS DE ÁGUA E ESGOTO DE NOVO HAMBURGO',
#                'DEP. DE ÁGUA E ESGOTOS - SANT\'ANA DO LIVRAMENTO',
#                'SERVIÇO AUTÔN. DE SANEAMENTO DE PELOTAS',
#                'CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RIO DOS SINOS',
#                'DEP. MUNICIPAL DE ÁGUAS E SANEAMENTO DE IJUÍ - DEMASI',
#                'FUND. MUNICIPAL DE SAÚDE DE CANOAS',
#                'FUND. MÉD.-HOSP. DR. HONOR T. COSTA - LAVRAS DO SUL',
#                'CIA. RIOGRANDENSE DE SANEAMENTO',
#                'DEPARTAMENTO DE ÁGUA, ARROIOS E ESGOTO DE BAGÉ',
#                'DEP. MUNICIPAL DE ÁGUA E ESGOTOS - PORTO ALEGRE',
#                'EMPRESA FARROUPILHENSE DE SANEAMENTO E DESENVOLVIMENTO AMBIENTAL S.A.',
#                'ÁGUAS DA NASCENTE - SERV. MUN. DE SANEAMENTO AMBIENTAL',
#                'ÁGUA DE IVOTI',
#                'TERMASA - CIA. ÁGUAS TERMAIS MARCELINO RAMOS',
#                'SEMAE - SERVIÇO MUN. DE ÁGUA E ESGOTOS - S. LEOPOLDO',
#                'DEMASI - DEP. MUNICIPAL DE ÁGUAS E SANEAMENTO DE IJUÍ',
#                'DMAE - DEP. MUNICIPAL DE ÁGUA E ESGOTOS - PORTO ALEGRE',
#                'DAEB - DEPARTAMENTO DE ÁGUA, ARROIOS E ESGOTO DE BAGÉ',
#                'DAE - DEP. DE ÁGUA E ESGOTOS - SANTANA DO LIVRAMENTO',
#                'SAMAE - SERVIÇO AUTÔN. MUN. DE ÁGUA E ESGOTO - CX.SUL',
#                ]

water_sewer = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["agua", "esgoto", "aguas", "esgotos", "saneamento"])
    )
).select(pl.col("NM_ORGAO_NLP"))


# Universities
universities = ['UNIVERSIDADE ESTADUAL DO RIO GRANDE DO SUL',
                'UERGS - UNIVERSIDADE ESTADUAL DO RIO GRANDE DO SUL',
                ]

# Intermunicipal Associations
intermunicipal_associations = df.filter(
    pl.col("NM_ORGAO_NLP").map_elements(
        lambda x: contains_substring(x, ["cons", "consorcio", "consorcios"])
    )
).select(pl.col("NM_ORGAO_NLP"))


# intermunicipal_associations = ['CONS. INTERM. DO VALE DO JACUÍ - CI/JACUÍ'
#               'CONS. PÚBL. INTERM. P/ ASSUNTOS ESTRATÉGICOS DO G8 - CIPAE G8',
#               'CONS. INTERM. - CI - CENTRO SUL',
#               'CONS. INTERM. ABRIGO INSTITUCIONAL DA PAZ - CIAIP',
#               'CONS. INTERM. DA REGIÃO CENTRO DO ESTADO DO RS - CI/CENTRO STA. MARIA',
#               'CONS. INTERM. DA REGIÃO NORDESTE',
#               'CONS. INTERM. DE COOP. EM GESTÃO PÚBLICA - CONIGEPU',
#               'CONS. INTERM. DE DESENV. SUSTENT. DA SERRA GAÚCHA - CISGA',
#               'CONS. INTERM. DE GESTÃO AMPLIADA DA REGIÃO CARBONÍFERA - CIGA',
#               'CONS. INTERM. DE SERVIÇOS DO VALE DO RIO PARDO',
#               'CONS. INTERM. DO MÉDIO ALTO URUGUAI',
#               'CONS. INTERM. DO VALE DO RIO CAÍ - CIS/CAÍ',
#               'CONS. INTERM. ESMERALDA / PINHAL DA SERRA',
#               'CONS. INTERM. LAR DE ACOLHIMENTO SÃO FRANCISCO',
#               'ADM. DE  CONSÓRCIOS  INTERMUNIC. S/A. - CAXIAS DO SUL',
#               'CONS. PÚBL. INTERM. DA REGIÃO DO ALTO URUGUAI - CIRAU',
#               'CONS. PÚBL. INTERM. DESENV. ECON., SOC. E AMB. MUN. BACIA RIO JAGUARÃO',
#               'CONS. DE DESENV. INTERM. DOS MUNIC. DO ALTO JACUÍ',
#               'CONS. DE DESENV. SUSTENT. DA QUARTA COLÔNIA - CONDESUS/QUARTA COLÔNIA',
#               'CONS. PÚBL. DA ASSOC. DOS MUNIC. DA REGIÃO METR. DE POA - CP GRANPAL',
#               'CONS. PÚBL. DO EXTREMO SUL', 'CONS. PÚBL. FRONTEIRA NOROESTE',
#               'CONSÓRCIO INTERMUNICIPAL DA REGIÃO DO PLANALTO MÉDIO',
#               'CONSÓRCIO PÚBLICO DA ASSOCIAÇÃO DOS MUNICÍPIOS DO VALE DO RIO DOS SINO',
#               'CONSÓRCIO PÚBLICO INTERMUNICIPAL DE INCLUSÃO SOCIAL E PRODUTIVA',
#               'CI/CENTRO - CONS. INTERM. DA REGIÃO CENTRO  ESTADO DO RS - STA. MARIA',
#               'COMANDAÍ - CONS. INTERM. DE GERENCIAMENTO DE RESÍDUOS SÓLIDOS URBANOS',
#               'CIEPS - CONS. INTERM. ESMERALDA / PINHAL DA SERRA',
#               'CONIGEPU - CONS. INTERM. DE COOP. EM GESTÃO PÚBLICA',
#               'CONISA - CONS. INTERM. DE SAÚDE',
#               'CIMAU - CONS. INTERM. DO MÉDIO ALTO URUGUAI',
#              'CIGRES - CONS. INTERM. DE GESTÃO DE RESÍDUOS SÓLIDOS',
#              'CISA - CONS. INTERM. SAÚDE DO NOROESTE DO ESTADO DO RS',
#              'CONSISA - CONS. INTERM. DE SERVIÇOS DO VALE DO TAQUARI',
#              'CIGA - CONS. INTERM. DE GESTÃO AMPLIADA DA REGIÃO CARBONÍFERA',
#              'CI-CS - CONS. INTERM. - CI - CENTRO SUL',
#              'CIS/CAÍ - CONS. INTERM. DO VALE DO RIO CAÍ',
#              'CIRENOR - CONS. INTERM. DA REGIÃO NORDESTE',
#              'CISMISSÕES - CONS. INTERM. DE SAÚDE DA REGIÃO DAS MISSÕES',
#              'CISGS - CONS. INTERM. DE SAÚDE DO GRANDE SARANDI',
#              'CONDESUS - CONS. INTERM. DE DESENV. SUSTENT. CAMINHO DAS ORIGENS',
#              'CI/JACUÍ - CONS. INTERM. DO VALE DO JACUÍ',
#              'CONDESUS - CONS. INTERM. DE DESENV. SUSTENT.- REGIÃO CAMPOS CIMA SERRA',
#              'CITEGEM - CONS. INTERM. DE GESTÃO MULTIFUNCIONAL',
#              'CIAIP - CONS. INTERM. ABRIGO INSTITUCIONAL DA PAZ',
#              'CILASFRA - CONS. INTERM. LAR DE ACOLHIMENTO SÃO FRANCISCO',
#              'CRESU - CONS. INTERM. DE RESÍDUOS SÓLIDOS',
#              'SDR - SECRETARIA DE DESENVOLVIMENTO RURAL',
#              'CISGA - CONS. INTERM. DE DESENV. SUSTENT. DA SERRA GAÚCHA - CISGA',
#              'COIS - CONS. INTERM. DE SAÚDE',
#              'CISVALE - CONS. INTERM. DE SERVIÇOS DO VALE DO RIO PARDO',
#              'CIRAU - CONS. PÚBL. INTERM. DA REGIÃO DO ALTO URUGUAI - CIRAU',
#              'CISP - CONSÓRCIO PÚBLICO INTERMUNICIPAL DE INCLUSÃO SOCIAL E PRODUTIVA',
#              'COMAJA - CONS. DE DESENV. INTERM. DOS MUNIC. DO ALTO JACUÍ',
#              'CP GRANPAL - CONS. PÚBL. DA ASSOC. DOS MUNIC. DA REGIÃO METR. DE POA',
#              'CIPLAM - CONSÓRCIO INTERMUNICIPAL DA REGIÃO DO PLANALTO MÉDIO',
#              'CP AMLINOR - CONS. PÚBL. DA ASSOC. DOS MUNIC. DO LITORAL NORTE',
#              'YUCUMÃ - CONS. ROTA DO YUCUMÃ',
#              'CONSIM - CONSORCIO DE SAUDE INTER-MUNICIPAL',
#              'COMUNORS - CONSÓRCIO DE DESENV. DOS MUNICÍPIOS DO NORTE DO RS',
#              'CIPAE G8 - CONS. PÚBL. INTERM. P/ ASSUNTOS ESTRATÉGICOS DO G8',
#              'COFRON - CONS. PÚBL. FRONTEIRA NOROESTE',
#              'CP SINOS - CONS. PÚBLICO DA ASSOC. MUNICÍPIOS DO VALE RIO DOS SINOS',
#              'CONDESUS/QUARTA COLÔNIA - CONS. DE DESENV. SUSTENT. DA QUARTA COLÔNIA',
#              'PRÓ-SINOS - CONS. PÚBL. SAN. BÁSICO DA BACIA HIDROG. DO RIO DOS SINOS',
#              'ADCOINTER - ADM. DE  CONSÓRCIOS  INTERMUNIC. S/A. - CAXIAS DO SUL',
#              'CPES - CONS. PÚBL. DO EXTREMO SUL',
#              'CONREPAR - CONS. REGIONAL DO PARANHANA',]


# Regulatory Agencies
regulatory_agencies = ['AGÊNCIA EST. DE REGULAÇÃO DOS SERV. PÚBL. DELEG. DO RGS',
                  'AGÊNCIA MUN. REGULAÇÃO  DOS SERV. PÚBL. DELEG. DE S. GABRIEL',
                  'AGÊNCIA REGULADORA DE SERVIÇOS PÚBLICOS DELEGADOS DE SANTA CRUZ DO SUL',
                  'AGÊNCIA REGULADORA DOS SERV. PÚBL. MUNIC. DE ERECHIM']

# Technology Agencies
technology_agencies = ['CIA. DE PROCESSAMENTO DE DADOS DO RS',
              'EMPRESA MUNICIPAL DE INFORMÁTICA DE PELOTAS',
              'CIA. PROC. DE DADOS DO MUNIC. PORTO ALEGRE']

# Cultural Organizations
cultural_organizations = ['FENAC S.A. FEIRAS E EMPREEND. TURÍSTICOS - N. HAMBURGO',
           'CIA. ÁGUAS TERMAIS MARCELINO RAMOS - TERMASA',
           'FESTA NACIONAL DA UVA TURISMO E EMPREENDIMENTOS S/A.',
           'FUND. CULT. AFIF JORGE SIMÕES FILHO - SÃO SEPÉ',
           'FUND. MUN. DE ARTES DE MONTENEGRO',
           'CIA. RIOGRANDENSE DE ARTES GRÁFICAS',
           'SEL - SECRETARIA DO ESPORTE E LAZER',
           'FUNDARC - FUND. MUN. DE ARTE E CULTURA - GRAVATAÍ',
           'FCAJSF - FUND. CULT. AFIF JORGE SIMÕES FILHO - SÃO SEPÉ',          
           ]

# Agricultural Institutions
agricultural_institutions = ['ASSOCIAÇÃO SULINA DE CRÉDITO E ASSISTÊNCIA RURAL',
               'CIA. ESTADUAL DE SILOS E ARMAZÉNS',
               'CIA. RIOGRANDENSE DE MINERAÇÃO',
               'INSTITUTO RIOGRANDENSE DO ARROZ',
               'FUND. MUN. DE MEIO AMBIENTE - GRAVATAÍ']

# State Agencies (Secretarias do Estado, Tribunais, Assembléia, etc)
state_agencies = ['PROCURADORIA-GERAL DE JUSTIÇA',
                     'SECRETARIA DA FAZENDA DO RS',
                     'SECRETARIA DA SEGURANÇA PÚBLICA',
                     'SECRETARIA DE JUSTIÇA, CIDADANIA E DIREITOS HUMANOS',
                     'SECRETARIA DO MEIO AMBIENTE E INFRAESTRUTURA',
                     'TRIBUNAL DE CONTAS DO ESTADO DO RS',
                     'TRIBUNAL DE JUSTIÇA MILITAR',
                     'DEFENSORIA PÚBLICA DO ESTADO',
                     'ASSEMBLEIA LEGISLATIVA']

# Other Institutions/Organizations
others = ['SUPERINTENDÊNCIA DO PORTO DE RIO GRANDE',
          'CENTRAL DE LICITAÇÕES DO ESTADO',
          'CENTRAIS DE ABASTECIMENTO DO RIO GRANDE DO SUL S.A.',
          'CIA. DE DESENV. DE CAXIAS DO SUL',
          'CIA. DE DESENV. DE PASSO FUNDO',
          'CIA. INDUSTRIAL E DE DESENV. URBANO - CRUZ ALTA',
          'INST. DE PLANEJAMENTO DE SANTA MARIA',
          'CIA. MUNICIPAL DE URBANISMO - NOVO HAMBURGO',
          'CODEPAS-P. - CIA. DE DESENV. DE PASSO FUNDO']

In [12]:
df_polars = df.with_columns(
    pl.when(pl.col("NM_ORGAO").is_in(city_halls)).then(pl.lit("city_halls"))
    .when(pl.col("NM_ORGAO").is_in(city_councils)).then(pl.lit("city_councils"))
    .when(pl.col("NM_ORGAO").is_in(police)).then(pl.lit("police"))
    .when(pl.col("NM_ORGAO").is_in(banks)).then(pl.lit("banks"))
    .when(pl.col("NM_ORGAO").is_in(social_security)).then(pl.lit("social_security"))
    .when(pl.col("NM_ORGAO").is_in(health_care)).then(pl.lit("health_care"))
    .when(pl.col("NM_ORGAO").is_in(energy)).then(pl.lit("energy"))
    .when(pl.col("NM_ORGAO").is_in(transportation)).then(pl.lit("transportation"))
    .when(pl.col("NM_ORGAO").is_in(water_sewer)).then(pl.lit("water_sewer"))
    .when(pl.col("NM_ORGAO").is_in(universities)).then(pl.lit("universities"))
    .when(pl.col("NM_ORGAO").is_in(intermunicipal_associations)).then(pl.lit("intermunicipal_associations"))
    .when(pl.col("NM_ORGAO").is_in(regulatory_agencies)).then(pl.lit("regulatory_agencies"))
    .when(pl.col("NM_ORGAO").is_in(technology_agencies)).then(pl.lit("technology_agencies"))
    .when(pl.col("NM_ORGAO").is_in(cultural_organizations)).then(pl.lit("cultural_organizations"))
    .when(pl.col("NM_ORGAO").is_in(agricultural_institutions)).then(pl.lit("agricultural_institutions"))
    .when(pl.col("NM_ORGAO").is_in(state_agencies)).then(pl.lit("state_agencies"))
    .when(pl.col("NM_ORGAO").is_in(others)).then(pl.lit("others"))   
    .otherwise(pl.lit("NAO_TEM")).alias("TP_ORGAO")
)


TypeError: invalid literal value: 'naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)\n\n SELECT [col("NM_ORGAO_NLP")] FROM\n  FILTER col("NM_ORGAO_NLP").map_list() FROM\n\n   WITH_COLUMNS:\n   [col("NM_ORGAO").map_list().map_list().map_list().map_list().alias("NM_ORGAO_NLP")]\n     WITH_COLUMNS:\n     [col("VL_HOMOLOGADO").fill_null([col("VL_LICITACAO")])]\n       WITH_COLUMNS:\n       [col("DT_HOMOLOGACAO").str.strptime([String(raise)]).alias("DT_HOMOLOGACAO")]\n         WITH_COLUMNS:\n         [col("VL_HOMOLOGADO").strict_cast(Float64)]\n           WITH_COLUMNS:\n           [col("ANO_LICITACAO").strict_cast(Int64)]\n             WITH_COLUMNS:\n             [col("CD_ORGAO").strict_cast(Int64)]\n               WITH_COLUMNS:\n               [col("ANO_LICITACAO").strict_cast(Int64)]\n                FILTER [(col("DS_OBJETO")) != (String(REGISTRO DE PREÇOS DE INSUMOS ))] FROM\n\n                 WITH_COLUMNS:\n                 [col("ANO_LICITACAO").replace([Ser

## Depois ver essa questao aqui dos 20 nao atribuidos

veja bem, o feature engineering pode ser o ultimo passo

In [12]:
df_polars.filter(pl.col("TP_ORGAO")=="NAO_TEM").collect().select("NM_ORGAO").unique().to_numpy()

array([['IPREV - INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DE VIAMÃO'],
       ['URUPREV - URUGUAIANA PREVIDENCIA SOCIAL'],
       ['STAS - SECRETARIA DE TRABALHO E ASSISTÊNCIA SOCIAL'],
       ['TERMASA - CIA. ÁGUAS TERMAIS MARCELINO RAMOS'],
       ['IRGA - INSTITUTO RIOGRANDENSE DO ARROZ'],
       ['SEL - SECRETARIA DO ESPORTE E LAZER'],
       ['CONREPAR - CONS. REGIONAL DO PARANHANA'],
       ['FUNDARC - FUND. MUN. DE ARTE E CULTURA - GRAVATAÍ'],
       ['FHGV - FUNDAÇÃO HOSPITALAR GETÚLIO VARGAS'],
       ['IGP - INSTITUTO-GERAL DE PERÍCIAS'],
       ['FCAJSF - FUND. CULT. AFIF JORGE SIMÕES FILHO - SÃO SEPÉ'],
       ['CODEPAS-P. - CIA. DE DESENV. DE PASSO FUNDO'],
       ['PREVIJUÍ - INST. PREV. SERV. PUBL. MUNIC. DE IJUÍ'],
       ['BANRISUL SOLUÇÕES EM PAGAMENTOS S.A. - INSTITUIÇÃO DE PAGAMENTO'],
       ['PASSOTUR - FUND. PASSO FUNDO DE TURISMO'],
       ['SETUR - SECRETARIA DE TURISMO'],
       ['CIDUSA - CIA. INDUSTRIAL E DE DESENV. URBANO - CRUZ ALTA'],
       ['ADCOINTER - 

## 3.2. **dt_homologacao_mes e dt_homologacao_semana_do_ano**

In [25]:
df_lic['dt_homologacao_mes'] = df_lic['dt_homologacao'].dt.month
df_lic['dt_homologacao_semana_do_ano'] = df_lic['dt_homologacao'].dt.isocalendar().week